In [48]:
#Import packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime as dt
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from arch import arch_model 

In [49]:
# Read CSV file
filename='D:\\2023semester\\Lund University\\Financial Risk Management\\lab2\\DataLab2.xlsx'
table = pd.read_excel(filename)
table['Date'] = pd.to_datetime(table['Date'])
table = table.set_index('Date')
table['losses'] = table['PL']*-1
estimation_data = table.iloc[0:500]
backtesting_sample = table.iloc[500:]
data_2007 = table[table.index.year == 2007]
data_2008 = table[table.index.year == 2008]
print(table.head())

             PL  VaRBHS  VaREWMA  VaRn  VaRt  VaRPot  ESEWMA  ESn  ESt  ESPot  \
Date                                                                            
2005-01-01  190     NaN      NaN   NaN   NaN     NaN     NaN  NaN  NaN    NaN   
2005-01-02  190     NaN      NaN   NaN   NaN     NaN     NaN  NaN  NaN    NaN   
2005-01-03 -220     NaN      NaN   NaN   NaN     NaN     NaN  NaN  NaN    NaN   
2005-01-05  120     NaN      NaN   NaN   NaN     NaN     NaN  NaN  NaN    NaN   
2005-01-06  450     NaN      NaN   NaN   NaN     NaN     NaN  NaN  NaN    NaN   

            losses  
Date                
2005-01-01    -190  
2005-01-02    -190  
2005-01-03     220  
2005-01-05    -120  
2005-01-06    -450  


In [50]:
#Kupiec
models = ['VaRBHS', 'VaREWMA', 'VaRn', 'VaRt', 'VaRPot']
alpha = 0.01

for model in models:
    n_violations_2007 = (data_2007['losses'] > data_2007[model]).sum()
    n_violations_2008 = (data_2008['losses'] > data_2008[model]).sum()
    
    expected_violations_2007 = alpha * len(data_2007)
    expected_violations_2008 = alpha * len(data_2008)
    
    pValKupiec_2007 = 1 - stats.binom.cdf(n_violations_2007 - 1, len(data_2007), alpha)
    pValKupiec_2008 = 1 - stats.binom.cdf(n_violations_2008 - 1, len(data_2008), alpha)
    
    
    print(f"Model: {model}")
    print(f"2007 VaR Exceptions: {n_violations_2007} Expected Number: {expected_violations_2007:.2f}, P-value: {pValKupiec_2007:.4f}, Result: {'Reject' if pValKupiec_2007 < alpha else 'Cannot Reject'}")
    print(f"2008 VaR Exceptions: {n_violations_2008} Expected Number: {expected_violations_2008:.2f}, P-value: {pValKupiec_2008:.4f}, Result: {'Reject' if pValKupiec_2008 < alpha else 'Cannot Reject'}")


Model: VaRBHS
2007 VaR Exceptions: 15 Expected Number: 2.52, P-value: 0.0000, Result: Reject
2008 VaR Exceptions: 7 Expected Number: 2.52, P-value: 0.0143, Result: Cannot Reject
Model: VaREWMA
2007 VaR Exceptions: 5 Expected Number: 2.52, P-value: 0.1105, Result: Cannot Reject
2008 VaR Exceptions: 6 Expected Number: 2.52, P-value: 0.0425, Result: Cannot Reject
Model: VaRn
2007 VaR Exceptions: 36 Expected Number: 2.52, P-value: 0.0000, Result: Reject
2008 VaR Exceptions: 12 Expected Number: 2.52, P-value: 0.0000, Result: Reject
Model: VaRt
2007 VaR Exceptions: 27 Expected Number: 2.52, P-value: 0.0000, Result: Reject
2008 VaR Exceptions: 9 Expected Number: 2.52, P-value: 0.0011, Result: Reject
Model: VaRPot
2007 VaR Exceptions: 5 Expected Number: 2.52, P-value: 0.1105, Result: Cannot Reject
2008 VaR Exceptions: 3 Expected Number: 2.52, P-value: 0.4620, Result: Cannot Reject


In [51]:
#Basel Traffic Light
#Focuses on one-sided Kupiec test
critical_value_amber = stats.binom.ppf(0.95, len(data_2007), alpha)
critical_value_red = stats.binom.ppf(0.9999, len(data_2008), alpha)

print(f"Amber Zone Critical Value: {critical_value_amber}")
print(f"Red Zone Critical Value: {critical_value_red}")

for model in models:
    n_violations_2007 = (data_2007['losses'] > data_2007[model]).sum()
    n_violations_2008 = (data_2008['losses'] > data_2008[model]).sum()
    
    if n_violations_2007 <= critical_value_amber:
        color_zone_2007 = "Green"
    elif critical_value_amber < n_violations_2007 <= critical_value_red:
        color_zone_2007 = "Amber"
    else:
        color_zone_2007 = "Red"
    
    if n_violations_2008 <= critical_value_amber:
        color_zone_2008 = "Green"
    elif critical_value_amber < n_violations_2008 <= critical_value_red:
        color_zone_2008 = "Amber"
    else:
        color_zone_2008 = "Red"
    
    print(f"Model: {model}")
    print(f"2007 Color Zone: {color_zone_2007}")
    print(f"2008 Color Zone: {color_zone_2008}")


Amber Zone Critical Value: 5.0
Red Zone Critical Value: 10.0
Model: VaRBHS
2007 Color Zone: Red
2008 Color Zone: Amber
Model: VaREWMA
2007 Color Zone: Green
2008 Color Zone: Amber
Model: VaRn
2007 Color Zone: Red
2008 Color Zone: Red
Model: VaRt
2007 Color Zone: Red
2008 Color Zone: Amber
Model: VaRPot
2007 Color Zone: Green
2008 Color Zone: Green


In [52]:
# Christoffersen Independence Test
for model in models:
    #2007
    data_2007['violations_' + model + '_2007'] = np.where(data_2007['losses'] > data_2007[model], 1, 0)
    n1_2007 = (data_2007['violations_' + model + '_2007'] == 1).sum()
    n0_2007 = len(data_2007) - n1_2007
    n11_2007 = 0
    n00_2007 = 0
    n01_2007 = 0
    n10_2007 = 0
    
    for r in range(0, len(data_2007) - 1):
        if data_2007['violations_' + model + '_2007'][r] == 1 and data_2007['violations_' + model + '_2007'][r + 1] == 1:
            n11_2007 += 1
        elif data_2007['violations_' + model + '_2007'][r] == 0 and data_2007['violations_' + model + '_2007'][r + 1] == 0:
            n00_2007 += 1
        elif data_2007['violations_' + model + '_2007'][r] == 0 and data_2007['violations_' + model + '_2007'][r + 1] == 1:
            n01_2007 += 1
        elif data_2007['violations_' + model + '_2007'][r] == 1 and data_2007['violations_' + model + '_2007'][r + 1] == 0:
            n10_2007 += 1
    
    pi00_2007 = n00_2007 / (n00_2007 + n01_2007)
    pi01_2007 = n01_2007 / (n00_2007 + n01_2007)
    pi10_2007 = n10_2007 / (n10_2007 + n11_2007)
    pi11_2007 = n11_2007 / (n10_2007 + n11_2007)
    pi0_2007 = n0_2007 / (n0_2007 + n1_2007)
    pi1_2007 = n1_2007 / (n0_2007 + n1_2007)

    lnNull_2007 = np.log(pi0_2007**n0_2007*pi1_2007**n1_2007)
    lnAlt_2007 = np.log(pi00_2007**n00_2007*pi01_2007**n01_2007*pi10_2007**n10_2007*pi11_2007**n11_2007)
    LRind_2007 = -2 * (lnNull_2007 - lnAlt_2007)
    pVal_2007 = 1 - stats.chi2.cdf(LRind_2007, 1)       
    
    #2008
    data_2008['violations_' + model + '_2008'] = np.where(data_2008['losses'] > data_2008[model], 1, 0)
    n1_2008 = (data_2008['violations_' + model + '_2008'] == 1).sum()
    n0_2008 = len(data_2008) - n1_2008
    n11_2008 = 0
    n00_2008 = 0
    n01_2008 = 0
    n10_2008 = 0
        
    for r in range(0, len(data_2008) - 1):
        if data_2008['violations_' + model + '_2008'][r] == 1 and data_2008['violations_' + model + '_2008'][r + 1] == 1:
            n11_2008 += 1
        elif data_2008['violations_' + model + '_2008'][r] == 0 and data_2008['violations_' + model + '_2008'][r + 1] == 0:
            n00_2008 += 1
        elif data_2008['violations_' + model + '_2008'][r] == 0 and data_2008['violations_' + model + '_2008'][r + 1] == 1:
            n01_2008 += 1
        elif data_2008['violations_' + model + '_2008'][r] == 1 and data_2008['violations_' + model + '_2008'][r + 1] == 0:
            n10_2008 += 1
    
    pi00_2008 = n00_2008 / (n00_2008 + n01_2008)
    pi01_2008 = n01_2008 / (n00_2008 + n01_2008)
    pi10_2008 = n10_2008 / (n10_2008 + n11_2008)
    pi11_2008 = n11_2008 / (n10_2008 + n11_2008)
    pi0_2008 = n0_2008 / (n0_2008 + n1_2008)
    pi1_2008 = n1_2008 / (n0_2008 + n1_2008)

    lnNull_2008 = np.log(pi0_2008**n0_2008*pi1_2008**n1_2008)
    lnAlt_2008 = np.log(pi00_2008**n00_2008*pi01_2008**n01_2008*pi10_2008**n10_2008*pi11_2008**n11_2008)
    LRind_2008 = -2 * (lnNull_2008 - lnAlt_2008)
    pVal_2008 = 1 - stats.chi2.cdf(LRind_2008, 1)
    
    print(f"Model: {model}")
    print(f"2007 LR_ind test statistic: {LRind_2007:.4f}, P-value: {pVal_2007:.4f}, Result: {'Reject' if pVal_2007 < alpha else 'Cannot Reject'} ")
    print(f"2008 LR_ind test statistic: {LRind_2008:.4f}, P-value: {pVal_2008:.4f}, Result: {'Reject' if pVal_2008 < alpha else 'Cannot Reject'} ")


Model: VaRBHS
2007 LR_ind test statistic: 1.3087, P-value: 0.2526, Result: Cannot Reject 
2008 LR_ind test statistic: 6.8231, P-value: 0.0090, Result: Reject 
Model: VaREWMA
2007 LR_ind test statistic: 9.9663, P-value: 0.0016, Result: Reject 
2008 LR_ind test statistic: 8.2158, P-value: 0.0042, Result: Reject 
Model: VaRn
2007 LR_ind test statistic: 5.5850, P-value: 0.0181, Result: Cannot Reject 
2008 LR_ind test statistic: 6.3087, P-value: 0.0120, Result: Cannot Reject 
Model: VaRt
2007 LR_ind test statistic: 5.9135, P-value: 0.0150, Result: Cannot Reject 
2008 LR_ind test statistic: 4.7217, P-value: 0.0298, Result: Cannot Reject 
Model: VaRPot
2007 LR_ind test statistic: 0.2434, P-value: 0.6217, Result: Cannot Reject 
2008 LR_ind test statistic: 5.4650, P-value: 0.0194, Result: Cannot Reject 


In [53]:
#ES Acerbi-Szekely (2015) test 
models = ['EWMA', 'n', 't', 'Pot']
alpha=0.01
for model in models:

    z_2007 = (-1)*(1/(len(data_2007)*alpha))*sum((data_2007['violations_' + 'VaR' + model + '_2007'] * data_2007['losses'] / data_2007['ES'+ model])) + 1
    z_2008 = (-1)*(1/(len(data_2008)*alpha))*sum((data_2008['violations_' + 'VaR' + model + '_2008'] * data_2008['losses'] / data_2008['ES'+ model])) + 1
    #alpha=0.01, so not use (1-alpha) to get estimated number of violation
    
    print(f"Model: ES{model}")
    print(f"2007 ES Acerbi-Szekely test statistic: {z_2007:.4f} ")
    print(f"2008 ES Acerbi-Szekely test statistic: {z_2008:.4f} ")


Model: ESEWMA
2007 ES Acerbi-Szekely test statistic: -0.4661 
2008 ES Acerbi-Szekely test statistic: -1.0487 
Model: ESn
2007 ES Acerbi-Szekely test statistic: -18.4620 
2008 ES Acerbi-Szekely test statistic: -6.5774 
Model: ESt
2007 ES Acerbi-Szekely test statistic: -1.3059 
2008 ES Acerbi-Szekely test statistic: 0.1105 
Model: ESPot
2007 ES Acerbi-Szekely test statistic: -1.1670 
2008 ES Acerbi-Szekely test statistic: -0.5021 


z closer to 0, the better the model,
So ESEWMA, ESt and ESPot seem a better choice.

In [54]:
#Simulate test statistics
degrees_of_freedom = [3, 5, 10, 'N(0,1)']
alpha = 0.01

critical_values_es = {}

for df in degrees_of_freedom:
    if df == 'N(0,1)':
        es_critical = stats.norm.ppf(1 - alpha)
    else:
        es_critical = stats.t.ppf(1 - alpha, df)
    
    critical_values_es[df] = -es_critical

print(f"Critical Values {critical_values_es}")


Critical Values {3: -4.540702858698419, 5: -3.3649299989072747, 10: -2.763769457447889, 'N(0,1)': -2.3263478740408408}


ESEWMA and ESn degree of freedom is infinite, so use critical value of -2.33;
ESt and ESPot degree of freedom near 3, so use critical value of -4.54;
So in the ES Acerbi-Szekely test, we cannot reject model ESEWMA, ESt and ESPot.

## Final Result:
a.
During the Kupiec test, BHS has bad performance in 2007 and good performance in 2008;
During the Christoffersen Independence Test, BHS has good performance in 2007 and bad performance in 2008;
Other models show the same performance for both 2007 and 2008;
So we can summarize for the VaR model: 
For VaRBHS, the method perform bad for both 2007 and 2008;
For VaREWMA, the method perform well for both 2007 and 2008;
For VaRn, the method perform bad for both 2007 and 2008;
For VaRt, the method perform bad for both 2007 and 2008;
For VaRPot, the method perform well for both 2007 and 2008;

We can also summarize for the ES model: 
For ESEWMA, the method perform well for both 2007 and 2008;
For ESn, the method perform bad for both 2007 and 2008;
For ESt, the method perform well for both 2007 and 2008;
For ESPot, the method perform well for both 2007 and 2008;

So generally, the same methods perform well or not so well for different sample years.



b.
During 2007 and 2008, the risk was increasingly high due to the financial crisis, so intuitively we should consider extreme situations, normal distribution may not be suitable.

We choose Pot method for VaR to evaluate extreme situations, as VaR focuses on the maximum potential loss.

We choose t distribution method for ES to focus on a fatter tail of losses, as ES is about the average loss that would be incurred if the portfolio falls above the VaR threshold, and choosing a different method compared with the VaR method of Pot can help us to gain a more comprehensive understanding of the risk. We do not choose the EMWA method because the volatility in 2007 and 2008 is not in cluster mode, and exhibits significant short-term variations
